In [1]:
import pickle
import os

In [2]:
with open('dic.pkl', 'rb') as f:
    dic = pickle.load(f)

In [3]:
lst = os.listdir('CompanyData')
orig = list(dic.keys())
chosen_list = []
delete_list = ['FALC', 'IDT', 'PRCP', 'BBOX', 'EVOL', 'CTS', 'FDC', 'NIHD', 'BRKS', 'KODK', 'OSPN', 'AEIS', 'LGL', 'QADA', 'PCMI', 'INVE', 'ZOOM', 'CACI']
for k in orig:
    if k in delete_list:
        pass
    else:
        chosen_list.append(k)
print(chosen_list)
with open('chosen_list.txt', 'w') as f:
    for item in chosen_list:
        f.write("%s\n" % item)
chosen_list = chosen_list[:-2]
n_of_comp = len(chosen_list)
n_of_comp

['PEGA', 'CASS', 'COHU', 'BDR', 'HMNY', 'BELFB', 'CDNS', 'KLAC', 'CY', 'XLNX', 'TACT', 'WGNR', 'NVDA', 'CMTL', 'AXE', 'TTWO', 'IT', 'INTC', 'ANEN', 'AMSWA', 'SYMC', 'AMKR', 'SPDC', 'CSCO', 'TLAB', 'SHEN', 'XRX', 'BHE', 'INTT', 'INAP', 'FTR', 'CRAY', 'SMIT', 'GVP', 'OLED', 'CYBE', 'MTSC', 'MCHP', 'ESCC', 'CTL', 'APH', 'LFUS', 'RMTR', 'VSAT', 'LSCC', 'ON', 'TDS', 'PAR', 'AAPL', 'PAYX', 'AXTI', 'EGOV', 'MMS', 'CELL', 'INTU', 'MSI', 'TRT', 'MSFT', 'QUIK', 'INOD', 'DGII', 'CSGS', 'DAIO', 'DDD', 'ASYS', 'WTT', 'EA', 'ITI', 'STCN', 'DIOD', 'NSYS', 'PLT', 'CYMI', 'MU', 'ZIXI', 'KVHI', 'NWK', 'DRCO', 'FICO', 'MSTR', 'IBM', 'CSPI', 'FSII', 'TYL', 'TXN', 'ADBE', 'HLIT', 'ALOT', 'CBB', 'PKE', 'SUPX', 'ADI', 'FARO', 'FFIV', 'EEFT', 'FLEX', 'VECO', 'MKSI', 'AWRE', 'FLIR', 'IAC', 'TSS', 'CIEN', 'TESS', 'MXWL', 'PRST', 'EBIX', 'IEC', 'VSH', 'ASUR', 'ZBRA', 'PCTI', 'TER', 'ANSS', 'TCX', 'EGHT', 'AMD', 'ACCL', 'WDC', 'OSIS', 'CREE', 'PFSW', 'ATEA', 'KLIC', 'NCR', 'CNXN', 'AVX', 'FEIM', 'ATNI', 'TRMB', '

130

In [4]:
print(len(dic['AAPL']['LAR']))

30


In [5]:
import numpy as np
def get_data(st, ed):
    X, Y = [], []
    for comp in chosen_list[st : ed]:
        for i in range(len(dic[comp]['sentiment_list']) - 1):
            X.append(list(dic[comp]['sentiment_list'][i]))
            #print(dic[comp]['LAR'][i])
            X[-1].append(dic[comp]['LAR'][i])
            X[-1].append(dic[comp]['ROA'][i])
            #X[-1] = np.array(X[-1])
            Y.append([dic[comp]['return_rate'][i]])
    X, Y = np.array(X), np.array(Y)
    return X, Y

In [6]:
from sklearn.linear_model import Ridge, LinearRegression
def train_linear_regression(X, Y, Xall, Yall):
    X = (X - np.mean(Xall, axis = 0)) / np.std(Xall, axis = 0)
    clf = Ridge(alpha = 0.1)
    clf.fit(X, Y)
    #print('training:')
    #print('L2 error:\t', np.sum((clf.predict(X) - Y) ** 2) / Y.shape[0])
    #print('score:\t', clf.score(X, Y))
    #print('validation:')
    #print('L2 error:\t', np.sum((clf.predict(Xval) - Yval) ** 2) / Yval.shape[0])
    #print('score:\t', clf.score(Xval, Yval))
    r = np.corrcoef(clf.predict(X).reshape(-1), Y.reshape(-1))
    rall = np.corrcoef(clf.predict(Xall).reshape(-1), Yall.reshape(-1))
    #print('\nCorrelated Coefficient:\n', r[0][1], rall[0][1])
    pred = []
    lenx = Xall.shape[0]
    for i in range(n_of_comp):
        Xind = (Xall[lenx //  n_of_comp * i: lenx // n_of_comp * (i + 1)] - np.mean(Xall, axis = 0)) / np.std(Xall, axis = 0)
        pred.append(clf.predict(Xind).reshape(-1))
    pred = np.array(pred)
    return clf.coef_, np.array(pred)

In [7]:
Xall, Yall = get_data(0, n_of_comp)
lenx = Xall.shape[0]
K = 13
pred = np.zeros((n_of_comp, 29))
coef = np.zeros((1, 10))
for i in range(K):
    X, Y = get_data(n_of_comp // K * i, n_of_comp // K * (i + 1))
    c, p = train_linear_regression(X, Y, Xall, Yall)
    pred += p
    coef += c
pred /= K
coef /= K
print(coef)
r = [np.corrcoef(pred[i], Yall[lenx // n_of_comp * i: lenx // n_of_comp * (i + 1)].reshape(-1))[0][1] for i in range(n_of_comp)]
corr_rank = [(r[i], chosen_list[i]) for i in range(n_of_comp)]
corr_rank = sorted(corr_rank, reverse = True)
print(np.mean(np.array(r), axis = None))
corr = {}
for i, j in corr_rank:
    corr[j] = i
print(len(corr_rank))
corr_rank

[[ 8.44602728e-05 -1.51054882e-05 -1.59574575e-04 -9.98358863e-05
  -5.47618318e-05 -1.57200524e-04 -6.50827796e-06 -9.06811036e-05
   1.62742127e-04 -2.23593889e-04]]
0.08534647886696178
130


[(0.6354217839216794, 'DRCO'),
 (0.5199553606500699, 'ANEN'),
 (0.4846815842227607, 'EEFT'),
 (0.48144212930863106, 'KVHI'),
 (0.43445205564994055, 'RMTR'),
 (0.41264556186710505, 'MSFT'),
 (0.40635817633446464, 'INTT'),
 (0.3765320044197491, 'AMKR'),
 (0.370484942149183, 'MTSC'),
 (0.3625135734819166, 'CY'),
 (0.3556036137947724, 'STCN'),
 (0.34392797505988576, 'IBM'),
 (0.3398903162940188, 'EGHT'),
 (0.3372150295090582, 'OSIS'),
 (0.3359023912226501, 'ESCC'),
 (0.3275705895487722, 'TESS'),
 (0.31572959044135573, 'PKE'),
 (0.3154316556473788, 'WDC'),
 (0.30509664989061264, 'PAR'),
 (0.30171399403984484, 'CASS'),
 (0.29789727223183066, 'SUPX'),
 (0.2886374150630895, 'EGOV'),
 (0.2855529254516693, 'ALOT'),
 (0.260063974639041, 'CSCO'),
 (0.2562423694918144, 'PFSW'),
 (0.2485119591978298, 'ASUR'),
 (0.2475519966533752, 'TTWO'),
 (0.246123072805334, 'ADI'),
 (0.24085384302536592, 'FTR'),
 (0.2398331225290606, 'VSH'),
 (0.2329499659107677, 'TDS'),
 (0.2295289241320267, 'FEIM'),
 (0.2234889

In [8]:
return_rank = sorted([(np.array(dic[comp]['return_rate']).mean(axis = None), comp) for comp in dic], reverse = True)
return_rate = {}
for i, j in return_rank:
    return_rate[j] = i
return_rank

[(0.00824511776536826, 'NWK'),
 (0.004138861046998415, 'IEC'),
 (0.0033789912658778744, 'ZOOM'),
 (0.0031451306713523833, 'HMNY'),
 (0.0023361385276890138, 'EGHT'),
 (0.001986836523966277, 'INOD'),
 (0.001912461784357913, 'RMTR'),
 (0.0018724905157980211, 'WGNR'),
 (0.0018457805679303002, 'CTSH'),
 (0.0018219940318812229, 'WDC'),
 (0.0017453636508853614, 'TTWO'),
 (0.0017371765943481566, 'FLIR'),
 (0.0017127286710416773, 'SMIT'),
 (0.0016866793500667668, 'DDD'),
 (0.0016829611487476426, 'GVP'),
 (0.0016248217736278463, 'INTT'),
 (0.0015333182902544202, 'EBIX'),
 (0.0015116431554015278, 'OSIS'),
 (0.0015036785648187747, 'DAIO'),
 (0.0014755432839198758, 'PFSW'),
 (0.0014682028938828987, 'CRAY'),
 (0.001432411169599108, 'SHEN'),
 (0.0014280463009426, 'TYL'),
 (0.001411448945149917, 'OLED'),
 (0.0014073593370485299, 'ASUR'),
 (0.0014046393907557722, 'ATNI'),
 (0.0014020704508548349, 'MTSC'),
 (0.0013962970054824133, 'ITI'),
 (0.0013679286465462417, 'ASYS'),
 (0.001361545850651471, 'CNXN')

In [11]:
bad_list = []
good_list = []
for comp in chosen_list:
    if corr[comp] < 0 and return_rate[comp] < 0:
        bad_list.append(comp)
for score, comp in corr_rank:
    if score > 0.4:
        good_list.append(comp)
print(good_list)
print(bad_list)
with open('good_list.pkl', 'wb') as f:
    pickle.dump(good_list, f)
with open('bad_list.pkl', 'wb') as f:
    pickle.dump(bad_list, f)
with open('return_rate.pkl', 'wb') as f:
    pickle.dump(return_rate, f)
with open('corr.pkl', 'wb') as f:
    pickle.dump(corr, f)
with open('chosen_list.pkl', 'wb') as f:
    pickle.dump(chosen_list, f)

['DRCO', 'ANEN', 'EEFT', 'KVHI', 'RMTR', 'MSFT', 'INTT']
['XRX', 'MKSI', 'TSS']


In [35]:
from matplotlib import pyplot as plt
for idx in range(130):
    X = pred[idx]
    Y = Yall[29 * idx: 29 * (idx + 1)].reshape(-1)
    plt.scatter(X, Y, color = 'blue')
    plt.plot(np.arange(-0.002, 0.003, 0.0001), np.poly1d(np.polyfit(X, Y, 1))(np.arange(-0.002, 0.003, 0.0001)), color = 'black')
    plt.xlim(-0.002, 0.003)
    plt.xlabel('optimistic score')
    plt.ylabel('market behavior')
    plt.gcf().subplots_adjust(left=0.2)
    plt.savefig('images_corr/{}.png'.format(chosen_list[idx]))
    plt.close(None)

In [36]:
for comp in chosen_list:
    X = list(range(29))
    Y1 = dic[comp]['orig_return_rate']
    Y2 = dic[comp]['msci']
    Y3 = dic[comp]['return_rate']
    plt.plot(X, Y1, label = 'company return rate', color = 'red')
    plt.plot(X, Y2, label = 'MSCI return rate', color = 'blue')
    plt.plot(X, Y3, label = 'relative return rate', color = 'green')
    plt.xlabel('t')
    plt.legend()
    plt.savefig('images_return_rate/{}.png'.format(comp))
    plt.close(None)

In [18]:
import requests
from bs4 import BeautifulSoup
comp_name = {}
for comp in chosen_list:
    r = requests.get("https://sec.report/Ticker/" + comp)
    soup = BeautifulSoup(r.text, "html.parser")
    
    comp_name[comp] = str(soup).split('Security and Exchange Commission filings for ')[1].split(' (')[0]
    print(comp_name[comp])
with open('comp_name.pkl', 'wb') as f:
    pickle.dump(comp_name, f)

PEGASYSTEMS INC
CASS INFORMATION SYSTEMS INC
COHU INC
BLONDER TONGUE LABORATORIES INC
Helios &amp; Matheson Analytics Inc.
BEL FUSE INC /NJ
CADENCE DESIGN SYSTEMS INC
KLA TENCOR CORP
CYPRESS SEMICONDUCTOR CORP /DE/
XILINX INC
TRANSACT TECHNOLOGIES INC
WEGENER CORP
NVIDIA CORP
COMTECH TELECOMMUNICATIONS CORP /DE/
ANIXTER INTERNATIONAL INC
TAKE TWO INTERACTIVE SOFTWARE INC
GARTNER INC
INTEL CORP
ANAREN  INC
AMERICAN SOFTWARE INC
SYMANTEC CORP
AMKOR TECHNOLOGY, INC.
Speed Commerce, Inc.
CISCO SYSTEMS, INC.
TELLABS INC
SHENANDOAH TELECOMMUNICATIONS CO/VA/
XEROX CORP
BENCHMARK ELECTRONICS INC
INTEST CORP
Internap Corp
FRONTIER COMMUNICATIONS CORP
CRAY INC
SCHMITT INDUSTRIES INC
GSE SYSTEMS INC
UNIVERSAL DISPLAY CORP \PA\
CYBEROPTICS CORP
MTS SYSTEMS CORP
MICROCHIP TECHNOLOGY INC
EVANS &amp; SUTHERLAND COMPUTER CORP
CENTURYLINK, INC
AMPHENOL CORP /DE/
LITTELFUSE INC /DE
RAMTRON INTERNATIONAL CORP
VIASAT INC
LATTICE SEMICONDUCTOR CORP
ON SEMICONDUCTOR CORP
TELEPHONE &amp; DATA SYSTEMS INC /DE